In [425]:
"""
Скрипт для обработки общйе таблицы Омского ЦОПП. 
"""

'\nСкрипт для обработки общйе таблицы Омского ЦОПП. \n'

In [426]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import tkinter
import sys
import os
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import time

pd.options.mode.chained_assignment = None  # default='warn'
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import copy
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import random

In [427]:
def check_sameness_range(df:pd.DataFrame,names_columns:list,border,amendment):
    
    """    
    Функция для проверки единнобразия значений в колонке по диапазонам
    """
    # получаем поправки для того чтобы диапазон ошибки указывался корректно
    first_correct = amendment[0]
    second_correct = amendment[1] 
    _error_df = pd.DataFrame(columns=['Строка или колонка с ошибкой', 'Описание ошибки']) # датафрейм для ошибок
    for column in names_columns:
        _temp_set = set(df[column].tolist())
        if len(_temp_set) != 1:
            temp_error_df = pd.DataFrame(data=[[f'Диапазон строк {border + first_correct} - {border+second_correct},Колонка {column}',
                                        'В указанном диапазоне и колонке  должны быть одинаковые значения во избежание ошибок при подсчете,если указан диапазон вне таблицы то удалите строки этого диапазона']],
                                 columns=['Строка или колонка с ошибкой',
                                          'Описание ошибки'])
            _error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
    return _error_df
        

In [428]:
def check_error_omsk(df:pd.DataFrame,border,amendment):
    """
    Функция для проверки на ошибки
    """

    error_df = pd.DataFrame(columns=['Строка или колонка с ошибкой', 'Описание ошибки']) # датафрейм для ошибок
    quantity_range = df.shape[0] // 5 #  получаем количество проходов которые нужно пройти
    sameness_columns = ['1','2','3'] # список колонок которые нужно проверить на единообразие внутри диапазона
    for i in range(quantity_range):
        temp_df = df.iloc[border:border+5,:]
        error_sameness_df = check_sameness_range(temp_df.copy(),sameness_columns,border,amendment) # проверка на единнообразие
        error_df = pd.concat([error_df, error_sameness_df], axis=0, ignore_index=True)
        
        
        
        
        border += 5
        
        
        
    
    
    
    
    
    return error_df
        
        
        
        
    
    
    
    

In [429]:
path_data_omsk = 'data/Омск/Трудоустройство_1_07_2023_выпуск_2022.xlsx'
path_to_end_folder = 'data'
# создаем датафрейм для регистрации ошибок
error_df = pd.DataFrame(columns=['Строка или колонка с ошибкой', 'Описание ошибки'])

In [430]:
# загружаем файл
df = pd.read_excel(f'{path_data_omsk}',dtype=str,skiprows=5)

In [431]:
df=df.iloc[:,:34] # убираем строки проверки

In [432]:

# заменяем названия колонок
df.columns = list(map(str,range(1,df.shape[1]+1)))

In [433]:
df.shape

(2030, 34)

In [434]:
# Проверяем на количество строк, должно быть кратно 5
if df.shape[0] % 5 !=0:
    temp_error_df = pd.DataFrame(data=[['',
                                        'Количество строк не кратно 5 !!! Возможно пропущены строки с данными или есть лишние строки в конце таблицы']],
                                 columns=['Строка или колонка с ошибкой',
                                          'Описание ошибки'])
    error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)

In [435]:
# Проверяем ошибки
tup_correct = (7, 11)  # создаем кортеж  с поправками
check_error_df= check_error_omsk(df.copy(),0,tup_correct)
error_df = pd.concat([error_df, check_error_df], axis=0, ignore_index=True)


In [436]:
error_df

,Строка или колонка с ошибкой,Описание ошибки
0,"Диапазон строк 142 - 146,Колонка 2",В указанном диапазоне и колонке должны быть о...


In [437]:
# создаем датафрейм из колонки 10 с названиями организаций и количеством работников
df_org = df[['1','2','3','4','5','9','34']]
df_org.fillna('Отсутствует',inplace=True)